## **Load libraries and data**

In [1]:
!pip install --upgrade pythainlp
!pip install pyLDAvis

Requirement already up-to-date: pythainlp in /usr/local/lib/python3.6/dist-packages (2.2.4)


In [2]:
import pandas as pd
import pythainlp
import gensim
import pyLDAvis.gensim
pyLDAvis.enable_notebook()
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 

In [3]:
df = pd.read_csv('Wongnai Reviews - Small.csv')

In [4]:
df.tail()

,Review ID,Review
295,296,ค่ำนี้คุณเพื่อนอยากส้มตำ หมูเฮาเลยพากันลงมากิน...
296,297,ร้านสะอาดดี ตกแต่งสวยงาม มีที่จอดรถ ราคาเมนูต่...
297,298,เช้าๆ รีบๆ วิ่งมาเข่าห้องเรียนแทบไม่ทันแต่ต้อง...
298,299,ร้านนี้เป็นร้านกาแฟเล็กๆ ข้างๆ ร้านๆ Happy Man...
299,300,ทรูคอฟฟี่สาขาซีคอนอยู่ในศูนย์บริการของทรู ชั้น...


## **Tokenize Words**

In [5]:
stopwords = list(pythainlp.corpus.thai_stopwords())
removed_words = [' ', '  ', '\n', 'ร้าน', '(', ')']
screening_words = stopwords + removed_words

def tokenize_with_space(sentence):
  merged = ''
  words = pythainlp.word_tokenize(str(sentence), engine='newmm')
  for word in words:
    if word not in screening_words:
      merged = merged + ',' + word
  return merged[1:]

In [6]:
df['Review_tokenized'] = df['Review'].apply(lambda x: tokenize_with_space(x))

In [7]:
df.tail()

,Review ID,Review,Review_tokenized
295,296,ค่ำนี้คุณเพื่อนอยากส้มตำ หมูเฮาเลยพากันลงมากิน...,"ค่ำ,เพื่อน,ส้มตำ,หมู,เฮา,ลงมา,กิน,ส้มตำ,ออฟฟิศ..."
296,297,ร้านสะอาดดี ตกแต่งสวยงาม มีที่จอดรถ ราคาเมนูต่...,"สะอาด,ดี,ตกแต่ง,สวยงาม,ที่จอดรถ,ราคา,เมนู,เทีย..."
297,298,เช้าๆ รีบๆ วิ่งมาเข่าห้องเรียนแทบไม่ทันแต่ต้อง...,"เช้า,รีบ,วิ่ง,เข่า,ห้องเรียน,แทบ,ต้องหา,ของกิน..."
298,299,ร้านนี้เป็นร้านกาแฟเล็กๆ ข้างๆ ร้านๆ Happy Man...,"ร้านกาแฟ,Happy,Mango,อาทิตย์,นัด,เพื่อน,นั่ง,ค..."
299,300,ทรูคอฟฟี่สาขาซีคอนอยู่ในศูนย์บริการของทรู ชั้น...,"ทรู,คอ,ฟ,ฟี่,สาขา,ซีคอน,ศูนย์,บริการ,ทรู,ชั้น,..."


## **Create Dictionary**

In [8]:
documents = df['Review_tokenized'].to_list()
texts = [[text for text in doc.split(',')] for doc in documents]
dictionary = gensim.corpora.Dictionary(texts)

In [9]:
print(dictionary.token2id.keys())

dict_keys(['20', 'Macchiato', 'กาแฟ', 'กาแฟร้อน', 'กิน', 'คน', 'ครึ่ง', 'ความคิด', 'ชอบ', 'ดื่ม', 'ตอน', 'ทาน', 'นึง', 'บาท', 'ปริมาณ', 'มีความสุข', 'ลา', 'สั่ง', 'หนัก', 'หลังจากนี้', 'เข้าไป', 'เจอ', 'เดิน', 'เดิม', 'เป็นประจำ', 'เฟล', 'แก้ว', 'โบราณ', '-', 'Art', 'Coffee', 'Kasetsart', '^^', 'of', 'คาราเมล', 'คิ', 'ชา', 'ซ', 'ซี', 'ดี', 'นั่ง', 'น่ารัก', 'บรรยากาศ', 'พนักงาน', 'พูดจา', 'มัค', 'รสชาติ', 'ราคา', 'ร้านกาแฟ', 'ลอน', 'ลาเต้', 'ลูกค้า', 'อัธยาศัย', 'อัสสัม', 'อา', 'อิตาเลียน', 'อุดหนุน', 'เพรส', 'เมนู', 'เยี่ยม', 'เอ๊กซ์', 'แพง', 'โซดา', 'โซ่', 'โต้', 'ไอ', 'ไอซ์', '555', 'กก', 'กรอบ', 'กระ', 'กระทะ', 'กร๊อบ', 'กลิ่น', 'กวง', 'กะ', 'กินน้ำ', 'กุ้ง', 'ก้น', 'ข้าวผัด', 'ข้าวเหนียว', 'ความสนใจ', 'คอ', 'คาว', 'งั้น', 'งี้', 'จริ๊ง', 'จัดจ้าน', 'จาน', 'จี๊ดจ๊าด', 'จ้น', 'จ้ิม', 'จ๊อบ', 'ฉ่า', 'ชิม', 'ชุ่มคอ', 'ดาว', 'ตบท้าย', 'ตัก', 'ถูกใจ', 'ทอด', 'ทะเล', 'ทีหลัง', 'นานา', 'นุ่ม', 'น้ำ', 'น้ำปลา', 'บรรยาย', 'ปลา', 'ปลาหมึก', 'ปาก', 'ปู', 'ปูม้า', 'ผสม', 'ผัด', 'พง', 'พริก', '

In [10]:
gensim_corpus = [dictionary.doc2bow(text, allow_update=True) for text in texts]
word_frequencies = [[(dictionary[id], frequence) for id, frequence in couple] for couple in gensim_corpus]

## **Topic Modeling**

In [11]:
num_topics = 30
chunksize = 4000 # size of the doc looked at every pass
passes = 20 # number of passes through documents
iterations = 50
eval_every = 1  # Don't evaluate model perplexity, takes too much time.

# Make a index to word dictionary.
temp = dictionary[0]  # This is only to "load" the dictionary.
id2word = dictionary.id2token

%time model = gensim.models.LdaModel(corpus=gensim_corpus, id2word=id2word, chunksize=chunksize, \
                       alpha='auto', eta='auto', \
                       iterations=iterations, num_topics=num_topics, \
                       passes=passes, eval_every=eval_every)

CPU times: user 16.4 s, sys: 11.4 s, total: 27.7 s
Wall time: 14.2 s


In [12]:
pyLDAvis.gensim.prepare(model, gensim_corpus, dictionary)

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
10    -0.020479 -0.038482       1        1  23.110175
23     0.117310 -0.090139       2        1  14.231380
25    -0.035873 -0.032308       3        1   6.204226
8     -0.026963 -0.028057       4        1   6.197069
21     0.004103 -0.088052       5        1   6.171739
4      0.126214  0.062318       6        1   4.492713
2     -0.042294 -0.031239       7        1   4.012277
11    -0.071980  0.027379       8        1   3.757063
0      0.017446 -0.005571       9        1   3.241260
13    -0.030033 -0.026600      10        1   3.195941
5      0.033646 -0.004397      11        1   3.064778
22    -0.052060 -0.007337      12        1   2.971881
6     -0.016152  0.026281      13        1   2.111870
17     0.007963  0.040087      14        1   1.989780
9     -0.018075 -0.008599      15        1   1.850664
24    -0.002488  0.003585      16        1   1.593289
26     0.024632  0.026791      17        1   1.575915
18    -0.016945  0.028079      18        1   1.522983
1      0.003759  0.010614      19        1   1.358368
29     0.021020  0.004891      20        1   1.165220
3     -0.020674  0.005632      21        1   1.011787
28    -0.004700  0.035230      22        1   0.915736
19    -0.003164  0.011519      23        1   0.862665
15    -0.000213  0.013029      24        1   0.851953
16     0.010949  0.011401      25        1   0.750814
20    -0.000689  0.013335      26        1   0.648859
7     -0.006831  0.015938      27        1   0.408270
27     0.001722  0.011910      28        1   0.300809
14    -0.000192  0.005761      29        1   0.242119
12     0.001043  0.006999      30        1   0.188395, topic_info=       Term        Freq       Total Category  logprob  loglift
157   อร่อย  474.000000  474.000000  Default  30.0000  30.0000
46   รสชาติ  385.000000  385.000000  Default  29.0000  29.0000
39       ดี  340.000000  340.000000  Default  28.0000  28.0000
2      กาแฟ  299.000000  299.000000  Default  27.0000  27.0000
11      ทาน  384.000000  384.000000  Default  26.0000  26.0000
..      ...         ...         ...      ...      ...      ...
266    ปั่น    0.184515   55.512787  Topic30  -5.8849   0.5677
411      ดู    0.204768  142.450432  Topic30  -5.7807  -0.2705
835     แถม    0.174269   51.494002  Topic30  -5.9420   0.5858
148     หอม    0.189787  130.963545  Topic30  -5.8567  -0.2624
105     น้ำ    0.175717  113.049524  Topic30  -5.9337  -0.1923

[2176 rows x 6 columns], token_table=      Topic      Freq                                               Term
term                                                                    
3691      9  0.702164                                                   
1205      1  0.323429                                                  !
1205      2  0.161715                                                  !
1205      3  0.107810                                                  !
1205      7  0.053905                                                  !
...     ...       ...                                                ...
4651      1  0.892737                                                  •
4693      2  0.835739                                                 ••
3148     24  0.694890  …................................................
2129      6  0.465965                                                  ′
2130      6  0.466872                                                  ″

[4603 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[11, 24, 26, 9, 22, 5, 3, 12, 1, 14, 6, 23, 7, 18, 10, 25, 27, 19, 2, 30, 4, 29, 20, 16, 17, 21, 8, 28, 15, 13])

In [13]:
model.show_topic(1)

[('เค้ก', 0.014751856),
 ('อร่อย', 0.01075815),
 ('รสชาติ', 0.009799483),
 ('ทาน', 0.0094283),
 ('ดี', 0.0076978332),
 ('ชา', 0.00694221),
 ('ราคา', 0.0064805807),
 ('หอม', 0.006462373),
 ('หวาน', 0.006441255),
 ('กาแฟ', 0.0057419683)]

In [14]:
df['topics'] = df['Review_tokenized'].apply(lambda x: model.get_document_topics(dictionary.doc2bow(x.split(',')))[0][0])
df['score'] = df['Review_tokenized'].apply(lambda x: model.get_document_topics(dictionary.doc2bow(x.split(',')))[0][1])

In [15]:
df.tail()

,Review ID,Review,Review_tokenized,topics,score
295,296,ค่ำนี้คุณเพื่อนอยากส้มตำ หมูเฮาเลยพากันลงมากิน...,"ค่ำ,เพื่อน,ส้มตำ,หมู,เฮา,ลงมา,กิน,ส้มตำ,ออฟฟิศ...",13,0.996950
296,297,ร้านสะอาดดี ตกแต่งสวยงาม มีที่จอดรถ ราคาเมนูต่...,"สะอาด,ดี,ตกแต่ง,สวยงาม,ที่จอดรถ,ราคา,เมนู,เทีย...",10,0.992349
297,298,เช้าๆ รีบๆ วิ่งมาเข่าห้องเรียนแทบไม่ทันแต่ต้อง...,"เช้า,รีบ,วิ่ง,เข่า,ห้องเรียน,แทบ,ต้องหา,ของกิน...",23,0.247305
298,299,ร้านนี้เป็นร้านกาแฟเล็กๆ ข้างๆ ร้านๆ Happy Man...,"ร้านกาแฟ,Happy,Mango,อาทิตย์,นัด,เพื่อน,นั่ง,ค...",23,0.996443
299,300,ทรูคอฟฟี่สาขาซีคอนอยู่ในศูนย์บริการของทรู ชั้น...,"ทรู,คอ,ฟ,ฟี่,สาขา,ซีคอน,ศูนย์,บริการ,ทรู,ชั้น,...",24,0.994567
